In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

Using TensorFlow backend.


In [2]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

170500096/170498071 [==============================] - 784s 5us/step


In [6]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [7]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [8]:
weight_decay = 1e-4

In [9]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [10]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [11]:
#training
batch_size = 64

In [12]:
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
781/781 [==============================] - 26s 33ms/step - loss: 1.9697 - acc: 0.4149 - val_loss: 1.4696 - val_acc: 0.5469
Epoch 2/125
781/781 [==============================] - 23s 30ms/step - loss: 1.4285 - acc: 0.5716 - val_loss: 1.4606 - val_acc: 0.6278
Epoch 3/125
781/781 [==============================] - 23s 30ms/step - loss: 1.3590 - acc: 0.6161 - val_loss: 1.2616 - val_acc: 0.6592
Epoch 4/125
781/781 [==============================] - 23s 30ms/step - loss: 1.2393 - acc: 0.6504 - val_loss: 1.0395 - val_acc: 0.7013
Epoch 5/125
781/781 [==============================] - 23s 30ms/step - loss: 1.2141 - acc: 0.6704 - val_loss: 0.9787 - val_acc: 0.7123
Epoch 6/125
781/781 [==============================] - 21s 27ms/step - loss: 1.1661 - acc: 0.6875 - val_loss: 0.9976 - val_acc: 0.7244
Epoch 7/125
781/781 [==============================] - 21s 27ms/step - loss: 1.0612 - acc: 0.7146 - val_loss: 0.9631 - val_acc: 0.7402
Epoch 8/125
781/781 [==============================] - 

781/781 [==============================] - 22s 28ms/step - loss: 0.6115 - acc: 0.8415 - val_loss: 0.7086 - val_acc: 0.8218
Epoch 62/125
781/781 [==============================] - 22s 28ms/step - loss: 0.6088 - acc: 0.8433 - val_loss: 0.6335 - val_acc: 0.8441
Epoch 63/125
781/781 [==============================] - 21s 27ms/step - loss: 0.6110 - acc: 0.8417 - val_loss: 0.6492 - val_acc: 0.8386
Epoch 64/125
781/781 [==============================] - 22s 28ms/step - loss: 0.6045 - acc: 0.8430 - val_loss: 0.6133 - val_acc: 0.8419
Epoch 65/125
781/781 [==============================] - 22s 28ms/step - loss: 0.6039 - acc: 0.8447 - val_loss: 0.5886 - val_acc: 0.8549
Epoch 66/125
781/781 [==============================] - 24s 30ms/step - loss: 0.6036 - acc: 0.8442 - val_loss: 0.5943 - val_acc: 0.8556
Epoch 67/125
781/781 [==============================] - 22s 28ms/step - loss: 0.5999 - acc: 0.8442 - val_loss: 0.5834 - val_acc: 0.8614
Epoch 68/125
781/781 [==============================] - 22s 2

781/781 [==============================] - 21s 27ms/step - loss: 0.4138 - acc: 0.8937 - val_loss: 0.4876 - val_acc: 0.8783
Epoch 122/125
781/781 [==============================] - 21s 27ms/step - loss: 0.4057 - acc: 0.8933 - val_loss: 0.4747 - val_acc: 0.8805
Epoch 123/125
781/781 [==============================] - 21s 27ms/step - loss: 0.4083 - acc: 0.8929 - val_loss: 0.4567 - val_acc: 0.8858
Epoch 124/125
781/781 [==============================] - 21s 27ms/step - loss: 0.4098 - acc: 0.8925 - val_loss: 0.4637 - val_acc: 0.8839
Epoch 125/125
781/781 [==============================] - 21s 27ms/step - loss: 0.4101 - acc: 0.8926 - val_loss: 0.4861 - val_acc: 0.8760


In [13]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 

In [17]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 103us/step

Test result: 87.600 loss: 0.486
